In [2]:
#imports
import requests

In [3]:
#Set up requests (get token)

# Open the text file and read the API key
with open('../API_Keys/NOAA_Token.txt', 'r') as file:
    api_key = file.read().strip()  # .strip() removes any leading/trailing whitespace

url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/'

headers = {
    'token': api_key
}

#Define an API call function
def api_call(url, endpoint, headers, parameters):
    url += endpoint
    response = requests.get(url, headers=headers, params=parameters)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        print("API Called")
    else:
        print("Error:", response.status_code)
        data = response.status_code
    return data

def append_to_csv(file_path, data):
    """Function to append data to a CSV file."""
    with open(file_path, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        
        if file.tell() == 0:  # Check if file is empty to write headers
            writer.writeheader()
        
        writer.writerows(data)

In [5]:
#List stations in Minnesota
endpoint = '/stations/'

# Specify the dataset, station ID, and date range
parameters = {
    'locationid': 'FIPS:72',
    'limit': 1000, # max is 1000 for a single request
    'offset': 1
}

data = api_call(url, endpoint, headers, parameters)

API Called


In [6]:
data

{'metadata': {'resultset': {'offset': 1, 'count': 413, 'limit': 1000}},
 'results': [{'elevation': 598.9,
   'mindate': '1947-06-01',
   'maxdate': '2015-08-01',
   'latitude': 18.1469,
   'name': 'ACEITUNA WATER TREATMENT PLANT, US',
   'datacoverage': 0.9475,
   'id': 'COOP:660040',
   'elevationUnit': 'METERS',
   'longitude': -66.4919},
  {'elevation': 454.8,
   'mindate': '1931-01-01',
   'maxdate': '2015-11-01',
   'latitude': 18.17236,
   'name': 'ADJUNTAS 2 NW, US',
   'datacoverage': 0.9303,
   'id': 'COOP:660053',
   'elevationUnit': 'METERS',
   'longitude': -66.73831},
  {'elevation': 557.8,
   'mindate': '1970-01-01',
   'maxdate': '2015-11-01',
   'latitude': 18.1747,
   'name': 'ADJUNTAS SUBSTATION, US',
   'datacoverage': 0.9964,
   'id': 'COOP:660061',
   'elevationUnit': 'METERS',
   'longitude': -66.7977},
  {'elevation': 14.9,
   'mindate': '1900-01-01',
   'maxdate': '1966-12-01',
   'latitude': 17.96667,
   'name': 'AGUIRRE, US',
   'datacoverage': 0.9888,
   'id'

In [30]:
all_results = []  # List to store all results

while True:
    data = api_call(url, endpoint, headers, parameters)
    if data is not None and 'results' in data:
        all_results.extend(data['results'])  # Append results from current call
        
        # Check if more results are available
        if parameters['offset'] + parameters['limit'] - 1 < data['metadata']['resultset']['count']:
            parameters['offset'] += parameters['limit']
        else:
            break
    else:
        # Handle error or no data case
        print("No data returned or an error occurred")
        break

# At this point, `all_results` contains all the stations fetched from the API

Error: 503


TypeError: argument of type 'int' is not iterable

In [7]:
all_results

[{'elevation': 276.5,
  'mindate': '1916-08-01',
  'maxdate': '2015-11-01',
  'latitude': 47.2991,
  'name': 'ADA, MN US',
  'datacoverage': 0.9497,
  'id': 'COOP:210018',
  'elevationUnit': 'METERS',
  'longitude': -96.5161},
 {'elevation': 348.1,
  'mindate': '1961-08-01',
  'maxdate': '2015-11-01',
  'latitude': 48.3005,
  'name': 'AGASSIZ REFUGE, MN US',
  'datacoverage': 0.9295,
  'id': 'COOP:210050',
  'elevationUnit': 'METERS',
  'longitude': -95.9816},
 {'elevation': 370.3,
  'mindate': '1948-08-01',
  'maxdate': '2015-11-01',
  'latitude': 46.5257,
  'name': 'AITKIN 2 E, MN US',
  'datacoverage': 0.8973,
  'id': 'COOP:210059',
  'elevationUnit': 'METERS',
  'longitude': -93.6674},
 {'elevation': 374.9,
  'mindate': '1931-01-01',
  'maxdate': '2015-11-01',
  'latitude': 43.6064,
  'name': 'ALBERT LEA 3 SE, MN US',
  'datacoverage': 0.9922,
  'id': 'COOP:210075',
  'elevationUnit': 'METERS',
  'longitude': -93.3019},
 {'elevation': 374.9,
  'mindate': '1948-08-01',
  'maxdate': 

In [17]:
#Test downloading data from all stations
endpoint = '/data/'

#Get list of station codes for the Minnesota stations
parameters = {
    'datasetid': 'GHCND',
    'locationid': 'FIPS:27',
    'datatypeid' : 'PRCP,TAVG',
    'limit': 1000, # max is 1000 for a single request
    'offset': 1,
    'startdate': '2023-12-30',
    'enddate': '2023-12-31'
}

data = api_call(url, endpoint, headers, parameters)
#data

API Called


In [18]:
data

{'metadata': {'resultset': {'offset': 1, 'count': 1353, 'limit': 1000}},
 'results': [{'date': '2023-12-30T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:CA006020559',
   'attributes': ',,C,',
   'value': 10},
  {'date': '2023-12-30T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:CA006020559',
   'attributes': ',,C,',
   'value': -40},
  {'date': '2023-12-30T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:US1MNAA0031',
   'attributes': ',,N,0700',
   'value': 0},
  {'date': '2023-12-30T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:US1MNAA0073',
   'attributes': ',,N,0700',
   'value': 0},
  {'date': '2023-12-30T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:US1MNAA0074',
   'attributes': ',,N,0433',
   'value': 0},
  {'date': '2023-12-30T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:US1MNAA0075',
   'attributes': ',,N,1615',
   'value': 0},
  {'date': '2023-12-30T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:US1MNAA0076',
   'att

In [5]:
#Test downloading data from all stations
endpoint = '/data/'

# Specify the dataset, station ID, and date range
parameters = {
    'datasetid': 'GHCND',
    'locationid': 'FIPS:27',
    'limit': 1000, # max is 1000 for a single request
    'offset': 1,
    'startdate': '2023-12-20',
    'enddate': '2023-12-20',
    'stationid': 'COOP:211065,COOP:218025,GHCND:US1MNMC0020,GHCND:USC00211065,GHCND:USC00218025'
}

data = api_call(url, endpoint, headers, parameters)
print(data)

all_results = []  # List to store all results

while True:
    data = api_call(url, endpoint, headers, parameters)
    if data is not None and 'results' in data:
        all_results.extend(data['results'])  # Append results from current call
        
        # Check if more results are available
        if parameters['offset'] + parameters['limit'] - 1 < data['metadata']['resultset']['count']:
            parameters['offset'] += parameters['limit']
        else:
            break
    else:
        # Handle error or no data case
        print("No data returned or an error occurred")
        break

# At this point, `all_results` contains all the stations fetched from the API

API Called
{'metadata': {'resultset': {'offset': 1, 'count': 6, 'limit': 1000}}, 'results': [{'date': '2023-12-20T00:00:00', 'datatype': 'PRCP', 'station': 'GHCND:USC00211065', 'attributes': ',,7,0800', 'value': 0}, {'date': '2023-12-20T00:00:00', 'datatype': 'SNOW', 'station': 'GHCND:USC00211065', 'attributes': ',,7,', 'value': 0}, {'date': '2023-12-20T00:00:00', 'datatype': 'SNWD', 'station': 'GHCND:USC00211065', 'attributes': ',,7,0800', 'value': 0}, {'date': '2023-12-20T00:00:00', 'datatype': 'TMAX', 'station': 'GHCND:USC00211065', 'attributes': ',,7,0800', 'value': 22}, {'date': '2023-12-20T00:00:00', 'datatype': 'TMIN', 'station': 'GHCND:USC00211065', 'attributes': ',,7,0800', 'value': -50}, {'date': '2023-12-20T00:00:00', 'datatype': 'TOBS', 'station': 'GHCND:USC00211065', 'attributes': ',,7,0800', 'value': -33}]}
API Called
